In [1]:
import os
os.environ['PATH'].split(';')

['C:\\Users\\mghan\\miniconda3',
 'C:\\Users\\mghan\\miniconda3\\Library\\mingw-w64\\bin',
 'C:\\Users\\mghan\\miniconda3\\Library\\usr\\bin',
 'C:\\Users\\mghan\\miniconda3\\Library\\bin',
 'C:\\Users\\mghan\\miniconda3\\Scripts',
 'C:\\Users\\mghan\\miniconda3\\bin',
 'C:\\Users\\mghan\\miniconda3\\Scripts\\condabin',
 'C:\\Users\\mghan\\bin',
 'C:\\Program Files\\Git\\mingw64\\bin',
 'C:\\Program Files\\Git\\usr\\local\\bin',
 'C:\\Program Files\\Git\\usr\\bin',
 'C:\\Program Files\\Git\\usr\\bin',
 'C:\\Program Files\\Git\\mingw64\\bin',
 'C:\\Program Files\\Git\\usr\\bin',
 'C:\\Users\\mghan\\bin',
 'C:\\gurobi911\\win64\\bin',
 'C:\\Program Files\\Rockwell Software\\RSCommon',
 'C:\\WINDOWS\\system32',
 'C:\\WINDOWS',
 'C:\\WINDOWS\\System32\\Wbem',
 'C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0',
 'C:\\WINDOWS\\System32\\OpenSSH',
 'C:\\Program Files\\WIDCOMM\\Bluetooth Software',
 'C:\\Program Files\\WIDCOMM\\Bluetooth Software\\syswow64',
 'C:\\Program Files\\Git\\cmd',
 'C:

# Model 1: Resource Assignment Problem (RAP) without any budget constraint

Consider a consulting company that has three open positions: Tester, Java Developer, and Architect. The three top candidates (resources) for the positions are: Carlos, Joe, and Monika. The consulting company administered competency tests to each candidate in order to assess their ability to perform each of the jobs. The results of these tests are called matching scores. Assume that only one candidate can be assigned to a job, and at most one job can be assigned to a candidate.

The problem is to determine an assignment of resources and jobs such that each job is fulfilled, each resource is assigned to at most one job, and the total matching scores of the assignments is maximized.

# Mathematical optimization
Mathematical optimization (which is also known as mathematical programming) is a declarative approach where the modeler formulates an optimization problem that captures the key features of a complex decision problem. 

A mathematical optimization model has five components:

    - Sets
    - Parameters
    - Decision variables
    - Constraints
    - Objective function(s)

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Input Data

In [2]:
# Resource and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# Matching score data
# Key: combination of R & J, Value: performance score 
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'):53, 
    ('Carlos', 'JavaDeveloper'):27,
    ('Carlos', 'Architect'): 13, 
    ('Joe', 'Tester'):80, 
    ('Joe', 'JavaDeveloper'):47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'):53, 
    ('Monika', 'JavaDeveloper'):73,
    ('Monika', 'Architect'): 47    
})

In [3]:
# Declare and initialize model
m1 = gp.Model('RAP1')

Academic license - for non-commercial use only - expires 2021-02-17
Using license file C:\Users\mghan\gurobi.lic


# Decision variables

To solve this assignment problem, we need to identify which resource is assigned to which job. We introduce a decision variable for each possible assignment of resources to jobs. Therefore, we have 9 decision variables.
For example,  $x_{2,1}$  is the decision variable associated with assigning the resource Joe to the job Tester. Therefore, decision variable  $x_{r,j}$  equals 1 if resource  $r \in R$  is assigned to job  $j \in J$ , and 0 otherwise.

In [4]:
# Create decision variables for the RAP model
# the vtype argument of the Model.addVars() method by default is non-negative and continuous
x = m1.addVars(combinations, name="assign")

In [5]:
x

{('Carlos', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Architect'): <gurobi.Var *Awaiting Model Update*>}

In [6]:
scores

{('Carlos', 'Tester'): 53,
 ('Carlos', 'JavaDeveloper'): 27,
 ('Carlos', 'Architect'): 13,
 ('Joe', 'Tester'): 80,
 ('Joe', 'JavaDeveloper'): 47,
 ('Joe', 'Architect'): 67,
 ('Monika', 'Tester'): 53,
 ('Monika', 'JavaDeveloper'): 73,
 ('Monika', 'Architect'): 47}

# Constraints

## Job constraints

These constraints need to ensure that each job is filled by exactly one resource.

For each job  $j \in J$ , take the summation of the decision variables over all the resources. We can write the corresponding job constraint as follows.

$$
\sum_{j=1}^{j=3} x_{r, 1} = 1\
$$

$$
\sum_{j=1}^{j=3} x_{r, 2} = 1\
$$

$$
\sum_{j=1}^{j=3} x_{r, 3} = 1\
$$

Or in general: 
$$
\sum_{r \in R } x_{r, j} = 1\ \forall j \in J
$$

In [7]:
# Create job constraints
# For each job j in the set of jobs J, equalize the summation of the decision variables over all the resources to 1 
jobs = m1.addConstrs((x.sum('*', j) == 1 for j in J), name='job') 

\$100

## Resource constraints

The constraints for the resources need to ensure that at most one job is assigned to each resource. That is, it is possible that not all the resources are assigned.

$$
\sum_{j=1}^{j=3} x_{1, j} \leqslant 1
$$

$$
\sum_{j=1}^{j=3} x_{2, j} \leqslant 1
$$

$$
\sum_{j=1}^{j=3} x_{3, j} \leqslant 1
$$

Or in general:
$$
\sum_{r \in R } x_{r, j} \leqslant 1 \hspace 1em \forall r \in R
$$

In [8]:
# Create resource constraints
# For each job j in the set of jobs J, equalize the summation of the decision variables over all the resources to 1 
resources = m1.addConstrs((x.sum(r, '*') <= 1 for r in R), name='resource') 

# Objective function

The goal is to miximize the total matching scores of the each assignments written as:

$$ 53x_{1,1}+80x_{2,1}+53x_{3,1} = \sum_{r \in R}s_{r,1}x_{r,1} $$
$$ 27x_{1,2}+47x_{2,2}+73x_{3,2} = \sum_{r \in R}s_{r,1}x_{r,2} $$
$$ 13x_{1,3}+63x_{2,3}+47x_{3,3} = \sum_{r \in R}s_{r,1}x_{r,3} $$

The objective function for matching scores of the assignments that satisfy the job and resource constraints can be concisely writen as:

$$ Maximize\sum_{j \in J}\sum_{r \in R} s_{r,j}x_{r,j} $$


In [9]:
# Objective: maximize total matching score of all assignments
m1.setObjective(x.prod(scores), GRB.MAXIMIZE)

In [10]:
# Save model for inspection
m1.write('RAP1.lp')

In [11]:
m1.display()

Maximize
   <gurobi.LinExpr: 53.0 assign[Carlos,Tester] + 27.0 assign[Carlos,JavaDeveloper] + 13.0 assign[Carlos,Architect] + 80.0 assign[Joe,Tester] + 47.0 assign[Joe,JavaDeveloper] + 67.0 assign[Joe,Architect] + 53.0 assign[Monika,Tester] + 73.0 assign[Monika,JavaDeveloper] + 47.0 assign[Monika,Architect]>
Subject To
   job[Tester] : <gurobi.LinExpr: assign[Carlos,Tester] + assign[Joe,Tester] + assign[Monika,Tester]> = 1.0
   job[JavaDeveloper] : <gurobi.LinExpr: assign[Carlos,JavaDeveloper] + assign[Joe,JavaDeveloper] + assign[Monika,JavaDeveloper]> = 1.0
   job[Architect] : <gurobi.LinExpr: assign[Carlos,Architect] + assign[Joe,Architect] + assign[Monika,Architect]> = 1.0
   resource[Carlos] : <gurobi.LinExpr: assign[Carlos,Tester] + assign[Carlos,JavaDeveloper] + assign[Carlos,Architect]> <= 1.0
   resource[Joe] : <gurobi.LinExpr: assign[Joe,Tester] + assign[Joe,JavaDeveloper] + assign[Joe,Architect]> <= 1.0
   resource[Monika] : <gurobi.LinExpr: assign[Monika,Tester] + assign[Mon

In [12]:
# Run optimization engine
m1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb6602fb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.930000000e+02


In [13]:
# Display optimal values of decison variables
for v in m1.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m1.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


# Model 2: Resource Assignment Problem with a budget constraint

Now, assume there is a fixed cost  Cr,j  associated with assigning a resource  r∈R  to job  j∈J . Assume also that there is a limited budget  B  that can be used for job assignments.

The cost of assigning Carlos, Joe, or Monika to any of the jobs is  $\$1,000$  ,  $\$2,000$  , and  $\$3,000$ respectively. The available budget is  $\$5,000$.

## Data

In [14]:
# Resources and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# Matching score data
# Cost is given in thousands of dollars

combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'):[27, 1], 
    ('Carlos', 'Architect'):[13, 1], 
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'):[47, 2], 
    ('Joe', 'Architect'):[67, 2], 
    ('Monika', 'Tester'): [53, 3],
    ('Monika', 'JavaDeveloper'):[73, 3], 
    ('Monika', 'Architect'):[47, 3]
})

# Available budget (thousands of solars)
budget = 5

In [15]:
m2 = gp.Model('RAP2')

# Decision variables

Because there is a budget constraint, it is possible that not all of the jobs will be filled. To account for this, we define a new decision variable that indicates whether or not a job is filled. Let  $g_{j}$  be equal 1 if job  $j \in J$  is not filled, and 0 otherwise. This variable is a gap variable that indicates that a job cannot be filled.

In [16]:
# Create decision variables for the RAP2 model (binary integer)
x = m2.addVars(combinations, vtype=GRB.BINARY, name="assign")

# create gap variables for the RAP2 model
g = m2.addVars(J, name='gap')

# Constraints

## Job constraints
limited budget to assign resources to jobs $ \rightarrow $ not all jobs are certainly filled
If for a particular job, the gap binary variable turns out to be 1 then that job is not filled. 
$$ \sum_{r \in R} x_{r, j} + g_{j} = 1 $$

In [17]:
# create job constraints
jobs = m2.addConstrs((x.sum('*', j) + g[j] == 1 for j in J), name='job')

## Resource constraints
$$ \sum_{r \in R } x_{r, j} \leqslant 1 $$

In [18]:
# create resource constraints
resources = m2.addConstrs((x.sum(r, '*') <= 1 for r in R), name='resource')

# Budget constraint

The total cost of filling the jobs should be less or equal than the budget B available (in thousand dollars):
$$ 
\left(1x_{1,1} + 2x_{2,1} + 3x_{3,1}\right) +
\left(1x_{1,2} + 2x_{2,2} + 3x_{3,2}\right) +
\left(1x_{1,3} + 2x_{2,3} + 3x_{3,3}\right) \leqslant 5  
$$

or in general format:
$$ \sum_{j \in J}\sum_{r \in R} C_{r,j}x_{r,j} \leqslant B $$

In [19]:
budget = m2.addConstr((x.prod(costs) <= budget), name='budget')

# Objective function
Objective function is: total matching scores of the assignments minus a penalty term! Why? It is possible that not all jobs are filled; `however, we want to heavily penalize this possibility as having gaps heavily deteriorates th etotal matching score value.` Since the maximum value of a matching score is 100, we give a big value of 101 to M: 

$$ Maximize\sum_{j \in J}\sum_{r \in R} s_{r,j}x_{r,j} - M\sum_{j \in J} g_{j} $$

In [20]:
# penalty for not filling a job position
M = 101

In [21]:
# Objective: maximize total matching score of assignments
# Unfilled jobs are heavily penalized
m2.setObjective(x.prod(scores) - M * g.sum(), GRB.MAXIMIZE)

In [22]:
m2.write('RAP2.lp')

In [23]:
m2.display()

Maximize
   <gurobi.LinExpr: 53.0 assign[Carlos,Tester] + 27.0 assign[Carlos,JavaDeveloper] + 13.0 assign[Carlos,Architect] + 80.0 assign[Joe,Tester] + 47.0 assign[Joe,JavaDeveloper] + 67.0 assign[Joe,Architect] + 53.0 assign[Monika,Tester] + 73.0 assign[Monika,JavaDeveloper] + 47.0 assign[Monika,Architect] + -101.0 gap[Tester] + -101.0 gap[JavaDeveloper] + -101.0 gap[Architect]>
Subject To
   job[Tester] : <gurobi.LinExpr: assign[Carlos,Tester] + assign[Joe,Tester] + assign[Monika,Tester] + gap[Tester]> = 1.0
   job[JavaDeveloper] : <gurobi.LinExpr: assign[Carlos,JavaDeveloper] + assign[Joe,JavaDeveloper] + assign[Monika,JavaDeveloper] + gap[JavaDeveloper]> = 1.0
   job[Architect] : <gurobi.LinExpr: assign[Carlos,Architect] + assign[Joe,Architect] + assign[Monika,Architect] + gap[Architect]> = 1.0
   resource[Carlos] : <gurobi.LinExpr: assign[Carlos,Tester] + assign[Carlos,JavaDeveloper] + assign[Carlos,Architect]> <= 1.0
   resource[Joe] : <gurobi.LinExpr: assign[Joe,Tester] + assign

In [24]:
# Run optimization engine
m2.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xf3c6f8c8
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0  121.66667    0    7   52.00000  121.66667   134%     -    0s
     0     0     cutoff    0        

In [25]:
# Display optimal values of decison variables
for v in m2.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Optimal objective function value: ', m2.objVal)

# Compute total matching score from assignment variables
total_matching_score = 0
for [r, j] in combinations:
    if x[r, j].x > 1e-6:
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score) 

assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
Optimal objective function value:  52.0
Total matching score:  153.0


# Model 3: Generalized RAP

In this sectiion, suppose, we have 200 hundred tasks in 6 different roles with a limited budget of $\$500,000$. Considering the budget limitation, can we fulfill all jobs or there would be gaps for some tasks? How much of budget is consumed and what is the maximum score we can obtain? There are the types of question we are going to answer. 

In [1]:
import gurobipy as gp
from gurobipy import GRB
import names 
import random
import numpy as np
from itertools import product

In [13]:
def generate_scenario(num_resources=100, num_jobs=100, roles=None,
                      score_mu=50, score_sigma=15, seed=10101):
    random.seed(seed)
    np.random.seed(seed)
    if roles is None:
        roles = {"Architect", "BackEndEngineer", "FrontEndEngineer",
                 "Tester", "DataScientist", "DataEngineer"}
    # P.D.F. of resource costs follows Benford's law, having support {1,2,...,9}
    # Benford's law states that in listings, tables of statistics, etc., the digit 1 tends to occur with probability ∼30%, 
    # much greater than the expected 11.1% (i.e., one digit out of 9). 
    # Benford's law can be observed, for instance, by examining tables of logarithms and noting that the first pages are much more worn and smudged than later pages. 
    benford = [np.log10((i+1)/i) for i in range(1,10)]
    # Sample resource names
    resources = {names.get_full_name(i) for i in range(num_resources)}
    # Sample job requirements, given that all roles are equally likely to be selected
    # multinomial arguments are numeber of experiments (jobs, here), probability of selecting a job and output shape
    req = np.random.multinomial(num_jobs, [1/len(roles)]*len(roles), size=1)[0] 
    jobs = set()
    # Assign ID to each job position
    for i, role in enumerate(roles):
        jobs = jobs.union(set(map(''.join, zip([role]*req[i], [str(x).zfill(int(np.log10(num_jobs))+1) for x in range(1,req[i]+1)]))))
    scores = {}
    costs = {}
    # Sample matching score and cost for each potential assignment
    for pair in product(resources, jobs):
        scores[pair] = int(np.clip(np.random.normal(score_mu, score_sigma), 0, 100))
        costs[pair] = random.choices(list(range(1,10)), weights=benford, k=1)[0]
    return resources, jobs, scores, costs 

# generate random resources, jobs, scores of each resource at each job and cost of assigning each resource to each job
res, job, ms, cst = generate_scenario() 

In [14]:
# assign budget
budget = 100

## Get a Greedy Solution (heuristic approach)

This simple greedy heuristic method follows the algorithm:

* consider the highest score,
* match the resource with the job of the highest score,
* eliminate the resource and the job from the table (recall the constraints that only one resource can be assigned to a job, and at most one job can be assigned to a resource)
* choose the next highest score if budgest is available and there is still scores to assign and repeat the process

In [4]:
def greedy_solve(resources, jobs, scores, costs, budget):
    assign = set()
    total_score = 0
    remaining_budget = budget
    # 200 resources and 200 jobs -> 200*200 scores combinations
    while remaining_budget > 0 and len(scores.keys()) > 0: # while there is still budget and remaining scores
        selection = max(scores, key=scores.get)
        assign.add(selection)
        total_score += scores[selection]
        remaining_budget -= costs[selection]
        # Remove potential assignments related to the resource/job of new selection
        res_filter = list(filter(lambda x: x[0] == selection[0], scores))
        job_filter = list(filter(lambda x: x[1] == selection[1], scores))
        blacklist = res_filter + job_filter
        scores = {key: val for key,val in scores.items()
                  if key not in blacklist
                  and costs[key] <= remaining_budget}
    print("Number of assignments: {0}".format(len(assign)))
    print("Total matching score: {0}".format(total_score))
    print("Budget consumed: {0}".format(budget - remaining_budget))
    
    kpi = {}
    kpi["n_assign"] = len(assign)
    kpi["total_ms"] = total_score
    kpi["budget_used"] = budget - remaining_budget
    return assign, kpi

In [5]:
greedy_sol, kpi = greedy_solve(res, job, ms, cst, budget)

# Greedy heuristic KPI's 
Greedy_assign = kpi["n_assign"]
Greedy_ms = kpi["total_ms"]

Number of assignments: 142
Total matching score: 13017
Budget consumed: 500


# Get Optimal Solution

In [16]:
m3 = gp.Model("RAP3")
assign = m3.addVars(ms.keys(), vtype=GRB.BINARY, name="assign")
g = m3.addVars(job, name="gap")
m3.addConstrs((assign.sum("*", j) + g[j]  == 1 for j in job), name="demand")
m3.addConstrs((assign.sum(r, "*") <= 1 for r in res), name="supply")
m3.addConstr(assign.prod(cst) <= budget, name="Budget")
BIGM = 101 # penatly of not filling a job 
m3.setObjective(assign.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)
m3.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 201 rows, 10100 columns and 30100 nonzeros
Model fingerprint: 0x86b93be9
Variable types: 100 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -10100.00000
Presolve time: 0.04s
Presolved: 201 rows, 10100 columns, 30100 nonzeros
Variable types: 0 continuous, 10100 integer (10100 binary)

Root relaxation: objective 7.732000e+03, 283 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7732.0000000 7732.00000  0.00%     -    0s

Explored 0 nodes (283 simplex iterations) in 0.10 seconds
Thread count was

In [35]:
m3.display()

Maximize
   <gurobi.LinExpr: 48.0 assign[Frank Smith,DataScientist2] + 39.0 assign[Frank Smith,BackEndEngineer1] + 48.0 assign[Frank Smith,Tester2] + 47.0 assign[Frank Smith,Tester1] + 66.0 assign[Frank Smith,DataScientist1] + 43.0 assign[Gaye Botner,DataScientist2] + 39.0 assign[Gaye Botner,BackEndEngineer1] + 57.0 assign[Gaye Botner,Tester2] + 55.0 assign[Gaye Botner,Tester1] + 34.0 assign[Gaye Botner,DataScientist1] + 22.0 assign[Garry Boston,DataScientist2] + 31.0 assign[Garry Boston,BackEndEngineer1] + 55.0 assign[Garry Boston,Tester2] + 57.0 assign[Garry Boston,Tester1] + 43.0 assign[Garry Boston,DataScientist1] + 51.0 assign[Melvin Gardner,DataScientist2] + 54.0 assign[Melvin Gardner,BackEndEngineer1] + 30.0 assign[Melvin Gardner,Tester2] + 49.0 assign[Melvin Gardner,Tester1] + 53.0 assign[Melvin Gardner,DataScientist1] + 39.0 assign[Karen Bartholomew,DataScientist2] + 58.0 assign[Karen Bartholomew,BackEndEngineer1] + 79.0 assign[Karen Bartholomew,Tester2] + 61.0 assign[Karen Ba

In [22]:
def print_solution(model):
    i = 1
    total_ms = 0
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}) {1}: {2}".format(i, var.varName, var.x))
            i += 1
            if "assign" in var.varName:
                total_ms += var.Obj
    print('Total matching score: {0}'.format(total_ms))
    print('Optimal objective function value: {0}'.format(model.objVal))
    return None

# display optimal values of decision variables
print_solution(m3)

1) assign[Robin Stanley,Tester01]: 1.0
2) assign[Belen Paradis,DataScientist04]: 1.0
3) assign[Melvin Gardner,DataScientist02]: 1.0
4) assign[Jane Moskowitz,FrontEndEngineer02]: 1.0
5) assign[Esther Miller,DataScientist05]: 1.0
6) assign[Ester Khoury,DataEngineer01]: 1.0
7) assign[Bob Green,BackEndEngineer02]: 1.0
8) assign[Tammy Quinn,BackEndEngineer01]: 1.0
9) assign[Hannah Hunt,BackEndEngineer03]: 1.0
10) assign[Charlotte Lewis,FrontEndEngineer01]: 1.0
11) assign[Michael Crawford,Architect01]: 1.0
12) assign[James Gibbons,Tester04]: 1.0
13) assign[Bruce Lamothe,DataScientist01]: 1.0
14) assign[Karen Bartholomew,DataEngineer02]: 1.0
15) assign[James Greve,Tester05]: 1.0
16) assign[Frank Smith,Tester02]: 1.0
17) assign[Gaye Botner,BackEndEngineer04]: 1.0
18) assign[Garry Boston,DataScientist03]: 1.0
19) assign[Earnest Blenman,FrontEndEngineer03]: 1.0
20) assign[Judith Martinez,Tester03]: 1.0
Total matching score: 1298.0
Optimal objective function value: 1298.0


In [33]:
# comparing KPI's of greedy heuristic and Gurobi Optimizer
Gurobi_assign = 0
Gurobi_ms = 0
for [r,j] in ms.keys():
    if (abs(assign[r, j].x) > 1e-6):
        Gurobi_assign = Gurobi_assign + assign[r, j].x
        Gurobi_ms = Gurobi_ms + ms[r, j]*assign[r, j].x
        
#print('Gurobi total assignments: ', Gurobi_assign)
#print('Gurobi total matching score: ', Gurobi_ms)

#Ratio of Greedy assignments respect to Gurobi assignments
assign_ratio = 100*Greedy_assign/Gurobi_assign
ms_ratio = 100*Greedy_ms/Gurobi_ms

print('Assignment ratio: ',"%.2f" % assign_ratio, '%')
print('Matching score ratio: ',"%.2f" % ms_ratio, '%')

Assignment ratio:  71.00 %
Matching score ratio:  74.12 %


## Analysis of Generalized RAP

    - In the heuristic approach, all budget is consumed while 58 jobs remain unallocated. Also, total maximum score obtained is 13017. 
    - Using optimization, all budget is depleted but no jobs remains unfilled and total maximum score is 17562. This is apart from the quickest run for binary integer optimization problem. 